### Load Libraries and Data

In [1]:
import pymc3 as pm
import theano
import theano.tensor as T
import numpy as np
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
import arviz as az
import lasagne
import os
import datetime
import re

floatX = theano.config.floatX
RANDOM_SEED = 42

In [2]:
# Constants (parse to log files)

ITER = 20000

In [3]:
def load_data():

    # Load Data

    df = pd.read_table("../data/rdata", header=None, delim_whitespace=True)
    df.columns = ["X", "Y"]
    df["index"] = np.where(df.index < 100, "Train", "Test")

    # Create train and test

    X_train = np.array(df.loc[df["index"] == "Train", "X"]).reshape(-1, 1)
    Y_train = np.array(df.loc[df["index"] == "Train", "Y"])
    X_test = np.array(df.loc[df["index"] == "Test", "X"]).reshape(-1, 1)
    Y_test = np.array(df.loc[df["index"] == "Test", "Y"])

    return X_train, X_test, Y_train, Y_test

In [4]:
X_train, X_test, Y_train, Y_test = load_data()

In [5]:
# Non-centered parametrization

n_hidden = [8]

# Initialize random weights between each layer

init_w_ih = np.zeros((X_train.shape[1], n_hidden[0])).astype(floatX)
init_w_ho = np.zeros((n_hidden[0], 1)).astype(floatX)
init_b_h = np.zeros((n_hidden[0], )).astype(floatX)
init_b_o = np.zeros((1, )).astype(floatX)

with pm.Model() as neural_network:

    bnn_input = theano.shared(X_train)
    bnn_output = theano.shared(Y_train)

    # ==== Hyperparameter Prior Definition (Precision)

    prec_w_ih = pm.Gamma("W_prec_ih", alpha=0.25, beta=0.000625, testval = 1)
    prec_w_ho = pm.Gamma("W_prec_ho", alpha=0.25, beta=7.8125e-05, testval = 1)
    prec_b_h = pm.Gamma("B_prec_h", alpha=0.25, beta=0.000625, testval = 1)
    prec_target = pm.Gamma("y_prec", alpha=0.25, beta=0.000625, testval = 1)

    # ==== Re-parametrized

    weights_ih_raw = pm.Normal("w_ih", mu=0, sigma=1, shape=(X_train.shape[1], n_hidden[0]), testval=init_w_ih)

    # Weights from hidden layer to output

    weights_ho_raw = pm.Normal("w_ho", mu=0, sigma=1, shape=(n_hidden[0], 1), testval=init_w_ho)

    # Hidden Biases
    
    biases_h_raw = pm.Normal("b_h", mu=0, sigma=1, shape=(n_hidden[0], ), testval=init_b_h)

    # Weights from input to hidden layer

    weights_ih = 1/np.sqrt(prec_w_ih) * weights_ih_raw

    # Weights from hidden layer to output

    weights_ho = (1/np.sqrt(prec_w_ho))*weights_ho_raw

    # Biases of hidden layer

    biases_h = 1/np.sqrt(prec_b_h)*biases_h_raw

    # Biases of output layer

    biases_o = pm.Normal("b_o", mu=0, sigma=100, shape=(1,), testval=init_b_o)

    # ==== Forward Pass of the Neural Network 

    # Build neural-network using Lasagne (keras equivalent)

    act_in = lasagne.layers.InputLayer(X_train.shape, input_var=bnn_input)

    act_h = lasagne.layers.DenseLayer(incoming=act_in, 
                                      num_units=n_hidden[0], 
                                      W=weights_ih,
                                      b=biases_h,    
                                      nonlinearity=lasagne.nonlinearities.tanh)

    act_out = lasagne.layers.DenseLayer(incoming=act_h, 
                                        num_units=1, 
                                        W=weights_ho, 
                                        b=biases_o)

    net_out = lasagne.layers.get_output(act_out)

    # Add likelihood function

    output = pm.Normal("output", net_out.flatten(), sigma=1/np.sqrt(prec_target), observed=bnn_output)

    # Sampling methods

    # step1 = pm.NUTS([prec_w_ih, prec_w_ho, prec_b_h, prec_target])
    # step2 = pm.NUTS([weights_ih, weights_ho, biases_h, biases_o])
    # steps = pm.CompoundStep([step1, step2])

In [6]:
# main.py

with neural_network:
    
    trace = pm.sample(draws=ITER, 
                      chains=4,
                      cores=4,
                      discard_tuned_samples=True,
                      return_inferencedata=False)

Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...
WARNING (theano.tensor.blas): We did not find a dynamic library in the library_dir of the library we use for blas. If you use ATLAS, make sure to compile it with dynamics library.
WARNING (theano.tensor.blas): We did not find a dynamic library in the library_dir of the library we use for blas. If you use ATLAS, make sure to compile it with dynamics library.
Multiprocess sampling (4 chains in 4 jobs)
NUTS: [b_o, b_h, w_ho, w_ih, y_prec, B_prec_h, W_prec_ho, W_prec_ih]


Sampling 4 chains for 1_000 tune and 20_000 draw iterations (4_000 + 80_000 draws total) took 3231 seconds.
There were 10611 divergences after tuning. Increase `target_accept` or reparameterize.
The acceptance probability does not match the target. It is 0.6455767592033724, but should be close to 0.8. Try to increase the number of tuning steps.
There were 2608 divergences after tuning. Increase `target_accept` or reparameterize.
The chain reached the maximum tree depth. Increase max_treedepth, increase target_accept or reparameterize.
There were 6834 divergences after tuning. Increase `target_accept` or reparameterize.
The chain reached the maximum tree depth. Increase max_treedepth, increase target_accept or reparameterize.
There were 6270 divergences after tuning. Increase `target_accept` or reparameterize.
The rhat statistic is larger than 1.4 for some parameters. The sampler did not converge.
The estimated number of effective samples is smaller than 200 for some parameters.


### Trace Plots

In [7]:
# Train predictions

with neural_network:
    ppc_train = pm.sample_posterior_predictive(
        trace, var_names=["output"], random_seed=42, samples=ITER,
    )

# Test predictions
    
bnn_input.set_value(X_test)
bnn_output.set_value(Y_test)

with neural_network:
    ppc_test = pm.sample_posterior_predictive(
        trace, var_names=["output"], random_seed=42, samples=ITER,
    )

/home/mjbajwa/anaconda3/lib/python3.6/site-packages/pymc3/sampling.py:1708: UserWarning:

samples parameter is smaller than nchains times ndraws, some draws and/or chains may not be represented in the returned posterior predictive sample



/home/mjbajwa/anaconda3/lib/python3.6/site-packages/pymc3/sampling.py:1708: UserWarning:

samples parameter is smaller than nchains times ndraws, some draws and/or chains may not be represented in the returned posterior predictive sample



In [8]:
df_predictions_train = pd.DataFrame({
    
    "inputs": X_train.flatten(),
    "targets": Y_train.flatten(),
    "mean": ppc_train["output"].mean(axis=0),
    "median": np.quantile(ppc_train["output"], 0.5, axis=0),
    "q1": np.quantile(ppc_train["output"], 0.01, axis=0),
    "q10": np.quantile(ppc_train["output"], 0.10, axis=0),
    "q90": np.quantile(ppc_train["output"], 0.90, axis=0),
    "q99": np.quantile(ppc_train["output"], 0.99, axis=0),
    "label": "train"

})


df_predictions_test = pd.DataFrame({
    
    "inputs": X_test.flatten(),
    "targets": Y_test.flatten(),
    "mean": ppc_test["output"].mean(axis=0),
    "median": np.quantile(ppc_test["output"], 0.5, axis=0),
    "q1": np.quantile(ppc_test["output"], 0.01, axis=0),
    "q10": np.quantile(ppc_test["output"], 0.10, axis=0),
    "q90": np.quantile(ppc_test["output"], 0.90, axis=0),
    "q99": np.quantile(ppc_test["output"], 0.99, axis=0),
    "label": "test"

})

df_predictions = pd.concat([df_predictions_train, df_predictions_test]).reset_index()

In [9]:
# Extract Traces

df_traces = pm.trace_to_dataframe(trace,
                                  chains=[0,1,2,3],
                                  varnames=["W_prec_ih", "W_prec_ho", "B_prec_h", "y_prec", "w_ih", "w_ho", "b_h", "b_o"])

df_traces["id"] = df_traces.index

df_traces["trace"] = np.where(np.logical_and(df_traces["id"] >= 0, df_traces["id"] < ITER), 1, 
                                  np.where(np.logical_and(df_traces["id"] >= ITER, df_traces["id"] < 2*ITER), 2, 
                                      np.where(np.logical_and(df_traces["id"] >= 2*ITER, df_traces["id"] < 3*ITER), 3, 
                                              np.where(np.logical_and(df_traces["id"] >= 3*ITER, df_traces["id"] <= 4*ITER), 4, 4))))

In [10]:
def divide_by_precision(df_traces, primary, precision):
    
    df_traces[primary] = df_traces[primary].div(np.sqrt(df_traces[precision]), axis=0)
    
    return df_traces

w_ih_cols = [col for col in df_traces.columns if re.match("w_ih_", col)]
b_h_cols = [col for col in df_traces.columns if re.match("b_h_", col)]
w_ho_cols = [col for col in df_traces.columns if re.match("w_ho_", col)]
b_o_cols = [col for col in df_traces.columns if re.match("b_o_", col)]

df_traces = divide_by_precision(df_traces, w_ih_cols, "W_prec_ih")
df_traces = divide_by_precision(df_traces, b_h_cols, "B_prec_h")
df_traces = divide_by_precision(df_traces, w_ho_cols, "W_prec_ho")
df_traces[b_o_cols] = df_traces[b_o_cols]

In [11]:
# Write to disk

time = datetime.datetime.now().strftime('%Y_%m_%d_%H_%M_%S')
new_dir = os.path.join("../output/", "pymc3_nc_" + time)
os.mkdir(new_dir)
df_traces.to_feather(f"{new_dir}/df_traces.feather")
df_predictions.drop(f"index", axis=1).to_feather(f"{new_dir}/df_predictions.feather")

In [12]:
print(new_dir)

../output/pymc3_nc_2021_03_23_13_41_24
